# **CNN 실습**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

# To prevent CCUBLAS_STATUS_ALLOC_FALED problem in tensorflow 2, the folling codes are necessary.
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        # 현재 메모리 증가량은 GPU 전체에 걸쳐 동일해야 합니다.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    except RuntimeError as e :
        # Memory growth must be set before GPUs have been initialized
        # GPU를 초기화하기 전에 메모리 증가를 설정해야 합니다.
        print(e)

In [ ]:
# Step 1 : Data preparation

(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_labels = len(np.unique(y_train))

# One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshaping
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])

# normalizing
x_train = x_train / 255.
x_test = x_test / 255.

In [ ]:
print(x_train.shape, y_train.shape)
print(y_train.shape, y_test.shape)

(60000, 28, 28, 1) (60000, 10)
(60000, 10) (10000, 10)


In [ ]:
# Step 2 : Model Construction

# Hyperparameter setup
input_shape = [image_size, image_size, 1]
batch_size = 128
kernel_size = 3
pool_size = 2
dropout = 0.3
epochs = 20
filters = 64

# Model Construction using functional API

inputs = Input(shape=input_shape)
y = Conv2D(filters = filters, kernel_size = kernel_size, activation = 'relu')(inputs)
y = MaxPooling2D(pool_size = pool_size)(y)
y = Conv2D(filters = filters, kernel_size = kernel_size, activation = 'relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters, kernel_size = kernel_size, activation = 'relu')(y)

# image to vector before connecting to dense layer
y = Flatten()(y)

# dropout regularization
y = Dropout(dropout)(y)

# Connect to Final output layer
outputs = Dense(num_labels, activation = 'softmax')(y)

# Build the model by supplying inputs/outputs
model = Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0   

In [ ]:
# Step 3 : Model Compile
model.compile(loss='categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
# Step 4 : Model Fit
import time
start_time = time.time()

model.fit(x_train, y_train,
          validation_data = (x_test, y_test),
          epochs = epochs,
          batch_size = batch_size)

print('Train Time in seconds : {:.2f}'.format(time.time() - start_time))

Epoch 1/20
469/469 [==============================] - 80s 168ms/step - loss: 0.2888 - accuracy: 0.9094 - val_loss: 0.0536 - val_accuracy: 0.9846
Epoch 2/20
469/469 [==============================] - 79s 168ms/step - loss: 0.0732 - accuracy: 0.9776 - val_loss: 0.0380 - val_accuracy: 0.9881
Epoch 3/20
469/469 [==============================] - 78s 167ms/step - loss: 0.0516 - accuracy: 0.9842 - val_loss: 0.0307 - val_accuracy: 0.9900
Epoch 4/20
469/469 [==============================] - 78s 167ms/step - loss: 0.0408 - accuracy: 0.9871 - val_loss: 0.0290 - val_accuracy: 0.9907
Epoch 5/20
469/469 [==============================] - 80s 171ms/step - loss: 0.0345 - accuracy: 0.9886 - val_loss: 0.0245 - val_accuracy: 0.9921
Epoch 6/20
469/469 [==============================] - 79s 168ms/step - loss: 0.0298 - accuracy: 0.9906 - val_loss: 0.0302 - val_accuracy: 0.9903
Epoch 7/20
469/469 [==============================] - 79s 167ms/step - loss: 0.0259 - accuracy: 0.9916 - val_loss: 0.0270 - val_ac

In [ ]:
# Step 5 : Model Evaluation
__, acc = model.evaluate(x_test, y_test, batch_size = batch_size)
print('Test Accuracy : {:.2f}%'.format(100*acc))

79/79 [==============================] - 3s 42ms/step - loss: 0.0257 - accuracy: 0.9931
Test Accuracy : 99.31%
